In [ ]:
from getpass import getpass
from json import loads
from os import environ
from pathlib import Path
from shlex import quote
from shutil import which
from subprocess import run
from typing import Any
from warnings import catch_warnings, filterwarnings

from pydantic import BaseModel, ConfigDict
from requests import get
from urllib3.exceptions import InsecureRequestWarning

SUNSHINE_CONFIG = "https://localhost:47990/api/config"
SUNSHINE_BW_ID = "sunshine"


def main():
    with catch_warnings():
        filterwarnings(action="ignore", category=InsecureRequestWarning)
        display(
            get(
                url=SUNSHINE_CONFIG,
                auth=tuple(get_bw(SUNSHINE_BW_ID).model_dump().values()),
                timeout=10,
                verify=False,  # noqa: S501
            ).json()
        )


class Login(BaseModel):
    model_config = ConfigDict(extra="ignore")
    username: str
    password: str


class BitwardenItem(BaseModel):
    model_config = ConfigDict(extra="ignore")
    login: Login


def get_bw(item: str) -> Login:  # sourcery skip: extract-method
    if not item:
        raise ValueError("Item required")
    email = input("BitWarden email")
    password = getpass("BitWarden master password")
    code = input("BitWarden 2FA code")
    try:
        result = run_bw("login", "--code", code, email, password)
        if not result["success"] and result["message"].startswith(
            "You are already logged in"
        ):
            run_bw("logout")
            if not result["success"]:
                raise RuntimeError(result["message"])
            result = run_bw("login", "--code", code, email, password)
        if not result["success"]:
            raise RuntimeError(result["message"])
        session = result["data"]["raw"]
        result = run_bw("--session", session, "get", "item", item)
        if not result["success"]:
            raise RuntimeError(result["message"])
        return BitwardenItem(**result["data"]).login
    finally:
        result = run_bw("logout")
        if not result["success"]:
            raise RuntimeError(result["message"])


def run_bw(*args: str) -> dict[str, Any]:
    bw_environ = {
        f"BW_{k}": v
        for k, v in {
            "CLEANEXIT": "true",
            "NOINTERACTION": "true",
            "PRETTY": "false",
            "QUIET": "false",
            "RAW": "true",
            "RESPONSE": "true",
        }.items()
    }
    original_bw_environ: dict[str, str] = {}
    for key, value in bw_environ.items():
        if environ.get(key):
            original_bw_environ[key] = environ[key]
        environ[key] = value
    try:
        return loads(run_npx("bw", *args))
    finally:
        for key in bw_environ:
            del environ[key]
        for key, value in original_bw_environ.items():
            environ[key] = value


def run_npx(*args: str) -> str:
    if node := which("node"):
        nodejs = Path(node).parent
    else:
        raise ValueError("Node not found. Is it installed?")
    return run(
        args=[
            arg.as_posix() if isinstance(arg, Path) else quote(arg)
            for arg in ["node", nodejs / "node_modules/npm/bin/npx-cli.js", *args]
        ],
        check=True,
        capture_output=True,
        encoding="utf-8",
    ).stdout


main()